In [ ]:
!pip3 install google-cloud-vision

In [ ]:
from google.cloud import vision
import io
import os
import pandas as pd
import numpy as np

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "natgeoanalytics-1df4edd54de3.json"

In [ ]:
df=pd.read_csv("data_sheet.csv")

In [ ]:
df['URL'][1]

'https://scontent-hou1-1.cdninstagram.com/v/t51.2885-15/307723625_160146663285741_3571520959207601833_n.jpg?stp=dst-jpg_e35_s1080x1080&_nc_ht=scontent-hou1-1.cdninstagram.com&_nc_cat=109&_nc_ohc=4lCsLS9Q1jkAX_fr0v3&edm=APU89FABAAAA&ccb=7-5&oh=00_AfBAm77nuS-cyVKXW1FpMv-mITPJFTY70AqS91r4LjI73Q&oe=65D68929&_nc_sid=bc0c2c'

In [ ]:
counter=0

def detect_labels_url(url):
    global counter
    label_list=''

    try:
      client = vision.ImageAnnotatorClient()
      image = vision.Image()
      image.source.image_uri = url

      response = client.label_detection(image=image)

      # Check for errors in the response
      if response.error.message:
        raise Exception(f'API Error: {response.error.message}')

      labels = response.label_annotations
      if labels:
          for label in labels:
            label_list+=label.description+', '


      else:
        print("No labels detected.")



    except:
      pass

    counter+=1
    return label_list.rstrip(' ').rstrip(',')

In [ ]:
df['Label']=np.nan
df.head()

,Unnamed: 0,Number,Caption,Like,URL,Label
0,0,1,Photo by @estherhorvath | Moss-covered trees a...,97727,https://scontent-hou1-1.cdninstagram.com/v/t51...,NaN
1,1,2,"Photo by @beverlyjoubert | My, what teeth you ...",195558,https://scontent-hou1-1.cdninstagram.com/v/t51...,NaN
2,2,3,Photo by @acacia.johnson | The first rays of a...,121855,https://scontent-hou1-1.cdninstagram.com/v/t51...,NaN
3,3,4,Photo by @jasperdoest | A white stork searches...,89130,https://scontent-hou1-1.cdninstagram.com/v/t51...,NaN
4,4,5,Photo by Robbie Shone @shonephoto | A cave sur...,55653,https://scontent-hou1-1.cdninstagram.com/v/t51...,NaN


In [ ]:
df['Label'] = df['URL'].apply(detect_labels_url)

In [ ]:
df['Label_len']=df['Label'].apply(len)

In [ ]:
df[df['Label_len']==0]

,Unnamed: 0,Number,Caption,Like,URL,Label,Label_len


In [ ]:
df['Caption_Pipe_Length']=df['Caption'].str.split('|').apply(len)

In [ ]:
df['Caption'].str.split('|').str[0]

0                               Photo by @estherhorvath 
1                              Photo by @beverlyjoubert 
2                              Photo by @acacia.johnson 
3                                 Photo by @jasperdoest 
4                     Photo by Robbie Shone @shonephoto 
                             ...                        
995    What do orcas, bonobos, and leaf-cutter ants h...
996             Photo by Prasenjeet Yadav @prasen.yadav 
997                               Photo by @joelsartore 
998    Experiencing the Northern Lights in all their ...
999    It’s been 18 years since Pluto’s celestial sta...
Name: Caption, Length: 1000, dtype: object

In [ ]:
df['Binary_Engagement']=df['Like']>df['Like'].median()

df['Binary_Engagement']=df['Binary_Engagement'].mask(df['Binary_Engagement']==True,1)

df['Binary_Engagement']=df['Binary_Engagement'].mask(df['Binary_Engagement']==False, 0)

In [ ]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Number,Caption,Like,URL,Label,Label_len,Binary_Engagement
0,0,0,1,Photo by @estherhorvath | Moss-covered trees a...,97727,https://scontent-hou1-1.cdninstagram.com/v/t51...,"Plant, Natural landscape, Terrestrial plant, W...",97,1
1,1,1,2,"Photo by @beverlyjoubert | My, what teeth you ...",195558,https://scontent-hou1-1.cdninstagram.com/v/t51...,"Plant, Plant community, Felidae, Carnivore, Ma...",93,1
2,2,2,3,Photo by @acacia.johnson | The first rays of a...,121855,https://scontent-hou1-1.cdninstagram.com/v/t51...,"Water, Nature, Slope, Natural landscape, Mount...",117,1
3,3,3,4,Photo by @jasperdoest | A white stork searches...,89130,https://scontent-hou1-1.cdninstagram.com/v/t51...,"Bird, Sky, Vertebrate, Nature, Extinction, Nec...",80,0
4,4,4,5,Photo by Robbie Shone @shonephoto | A cave sur...,55653,https://scontent-hou1-1.cdninstagram.com/v/t51...,"Flash photography, Outdoor recreation, Recreat...",131,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#prepare text_documents
text_documents = []
for label in df['Label']:
    concatenated_text = " ".join(label.split(', '))
    text_documents.append(concatenated_text)

#create TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(text_documents).toarray()
terms = tfidf_vectorizer.get_feature_names_out()

df_label_tfidf = pd.DataFrame(tfidf_matrix, columns=terms)

# Display the DataFrame
print(df_label_tfidf)

     abdomen  accessory  accipitridae  accipitriformes  adaptation  address  \
0        0.0        0.0           0.0              0.0    0.000000      0.0   
1        0.0        0.0           0.0              0.0    0.000000      0.0   
2        0.0        0.0           0.0              0.0    0.000000      0.0   
3        0.0        0.0           0.0              0.0    0.271832      0.0   
4        0.0        0.0           0.0              0.0    0.000000      0.0   
..       ...        ...           ...              ...         ...      ...   
995      0.0        0.0           0.0              0.0    0.000000      0.0   
996      0.0        0.0           0.0              0.0    0.000000      0.0   
997      0.0        0.0           0.0              0.0    0.000000      0.0   
998      0.0        0.0           0.0              0.0    0.000000      0.0   
999      0.0        0.0           0.0              0.0    0.000000      0.0   

     adventure  advertising  adã  aeolian  ...     

In [ ]:
#feature selection
from sklearn.feature_selection import SelectKBest, chi2

# Select the top 100 features based on chi-squared score
df_label_tfidf.fillna(0, inplace=True)
k = 100
tfidf_matrix_selected = SelectKBest(chi2, k=k).fit_transform(df_label_tfidf , df['Binary_Engagement'].to_list())
print(tfidf_matrix_selected.shape)

(1000, 100)


In [ ]:
# Importance of each word
fit = SelectKBest(score_func=chi2, k=k).fit(df_label_tfidf , df['Binary_Engagement'].to_list())

df_column_score = pd.DataFrame({'Column_name': df_label_tfidf.columns, 'P-value': fit.pvalues_})
# P-value of each word
print(df_column_score.sort_values(by='P-value',ascending = True).head(100))

    Column_name   P-value
343     felidae  0.004779
171   carnivore  0.007030
178        cats  0.013982
458         hat  0.015285
685    organism  0.018299
..          ...       ...
497          in  0.244451
688       otter  0.245318
516       jeans  0.245450
116      blazer  0.245512
94        beach  0.245653

[100 rows x 2 columns]


In [ ]:
#Logistic Regrssion
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_selected, df['Binary_Engagement'].to_list(), test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy:", accuracy)

Confusion Matrix:
[[80 27]
 [37 56]]

Accuracy: 0.68


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Tokenization and lowercase conversion
    tokens = nltk.word_tokenize(text.lower())
    # Remove punctuation and stopwords
    cleaned_tokens = [token for token in tokens if token not in stop_words]
    # Rejoin tokens into a clean text string
    cleaned_text = " ".join(cleaned_tokens)

    return cleaned_text


#prepare text_documents

text_documents = [clean_text(document) for document in df['Caption']]

#create TF-IDF matrix
#print(text_documents)

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(text_documents)
tfidf_array = tfidf_matrix.toarray()
terms = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame
df_caption_tfidf = pd.DataFrame(tfidf_array, columns=terms)
# Display the DataFrame
#df_caption_tfidf['Y'] = df['engagement']
#df_caption_tfidf.to_csv("caption.csv")
print(df_caption_tfidf)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     000  000th   03   10  100  102  108  109  10m   11  ...  zoroaster  \
0    0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...        0.0   
1    0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...        0.0   
2    0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...        0.0   
3    0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...        0.0   
4    0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...        0.0   
..   ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...        ...   
995  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...        0.0   
996  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...        0.0   
997  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...        0.0   
998  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...        0.0   
999  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...        0.0   

     zumbahua  zzz  ângela  åkernes  ålesund  élégantes  ívar  żabice  الاردن  
0         0.0  0.0 

In [ ]:
#feature selection

from sklearn.feature_selection import SelectKBest, chi2

# Select the top 100 features based on chi-squared score
df_caption_tfidf.fillna(0, inplace=True)
k = 100
tfidf_matrix_selected2 = SelectKBest(chi2, k=k).fit_transform(df_caption_tfidf , df['Binary_Engagement'].to_list())
print(tfidf_matrix_selected.shape)

(1000, 100)


In [ ]:
# Importance of each word
test = SelectKBest(score_func=chi2, k=k).fit(df_caption_tfidf , df['Binary_Engagement'].to_list())
#print(fit.pvalues_.shape)

df_column_score = pd.DataFrame({'Column_name': df_caption_tfidf.columns, 'P-value': test.pvalues_})
# P-value of each word
print(df_column_score.sort_values(by='P-value',ascending = True).head(100))


      Column_name   P-value
1841         cats  0.129829
690        animal  0.131415
1547  brianskerry  0.136981
1998        chocó  0.146108
3356      ecuador  0.162205
...           ...       ...
1833          cat  0.356757
8163        point  0.357184
3698      evolved  0.357754
6737       meteor  0.357952
8305    predators  0.359189

[100 rows x 2 columns]


In [ ]:

#Logistic Regrssion
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_selected2, df['Binary_Engagement'].to_list(), test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy:", accuracy)

Confusion Matrix:
[[77 30]
 [34 59]]

Accuracy: 0.68


In [ ]:
tfidf_label = pd.DataFrame(tfidf_matrix_selected)
tfidf_caption = pd.DataFrame(tfidf_matrix_selected2)

# Concatenate horizontally
combined_df = pd.concat([tfidf_label , tfidf_caption], axis=1)

combined_df.columns = range(len(combined_df.columns))

#print(combined_df)

#Logistic Regrssion
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(combined_df, df['Binary_Engagement'].to_list(), test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy:", accuracy)

Confusion Matrix:
[[78 29]
 [30 63]]

Accuracy: 0.705


In [ ]:
df.to_csv('data_sheet.csv')